# Removedor de Fundo

Baseado nesse [colab](https://colab.research.google.com/github/eyaler/avatars4all/blob/master/yarok.ipynb)

In [ ]:
#@title Instalar
#@markdown É só executar a célula e esperar o processo de instalação terminar
%cd /content
!git clone --depth 1 https://github.com/eyaler/U-2-Net
!mkdir -p /content/U-2-Net/saved_models/u2net
%cd /content/U-2-Net/saved_models/u2net
import os
if not os.path.exists('/content/U-2-Net/saved_models/u2net/u2net.pth'):
  !gdown https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O /content/U-2-Net/saved_models/u2net/u2net.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net.pth
if not os.path.exists('/content/U-2-Net/saved_models/u2net/u2netp.pth'):
  !gdown https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O /content/U-2-Net/saved_models/u2net/u2netp.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2netp.pth
!mkdir -p /content/U-2-Net/saved_models/u2net_portrait
%cd /content/U-2-Net/saved_models/u2net_portrait
if not os.path.exists('/content/U-2-Net/saved_models/u2net_portrait/u2net_portrait.pth'):
  !gdown https://drive.google.com/uc?id=1IG3HdpcRiDoWNookbncQjeaPN28t90yW -O /content/U-2-Net/saved_models/u2net_portrait/u2net_portrait.pth
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/u2net_portrait.pth
%cd /content
!git clone --depth 1 https://github.com/eyaler/MODNet
%cd MODNet/pretrained
if not os.path.exists('/content/MODNet/pretrained/modnet_photographic_portrait_matting.ckpt'):
  !gdown https://drive.google.com/uc?id=1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz -O /content/MODNet/pretrained/modnet_photographic_portrait_matting.ckpt
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/modnet_photographic_portrait_matting.ckpt
if not os.path.exists('/content/MODNet/pretrained/modnet_webcam_portrait_matting.ckpt'):
  !gdown https://drive.google.com/uc?id=1Nf1ZxeJZJL8Qx9KadcYYyEmmlKhTADxX -O /content/MODNet/pretrained/modnet_webcam_portrait_matting.ckpt
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/modnet_webcam_portrait_matting.ckpt
%cd /content
!pip install -U youtube-dl
!pip install -U imageio
!pip install -U imageio-ffmpeg

In [ ]:
#@title Selecione a imagem
#@markdown 1. No **foreground_url** você cola o link da imagem

#@markdown *Dica: Você pode usar o Discord, colocando as imagens em um canal e copiando o link, ou use o imgur.com*

#@markdown 2. Depois é só executar a célula


foreground_url = '' #@param {type:"string"}
background_url = 'Transparent' #@param ['Transparent']
import os
import youtube_dl
def is_supported(url):
    if url.lower().endswith(('.png','.jpg','.jpeg','.bmp')):
      return False
    extractors = youtube_dl.extractor.gen_extractors()
    for e in extractors:
        if e.suitable(url) and e.IE_NAME != 'generic':
            return True
    return False

if foreground_url:
  !rm -f /content/foreground
  if is_supported(foreground_url):
    !rm -f /content/foreground.mp4
    !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4' '$foreground_url' --merge-output-format mp4 -o /content/foreground
    !mv /content/foreground.mp4 /content/foreground 
    fg_time_params = ''
  if not os.path.exists('/content/foreground'):
    !wget '$foreground_url' -O /content/foreground

if '://' in background_url:
  !rm -f /content/background
  if is_supported(background_url):
    !rm -f /content/background.mp4
    !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01]+bestaudio[ext=m4a]/mp4' '$background_url' --merge-output-format mp4 -o /content/background
    !mv /content/background.mp4 /content/background
    bg_time_params = ''
  if not os.path.exists('/content/background'):
    !wget '$background_url' -O /content/background
else:
  background_url = background_url.lower()

In [ ]:
#@title Remover o fundo

#@markdown É só executar a célula que o fundo será removido

#@markdown ***Notas sobre os modelos de I.A***

#@markdown Você tem 5 modelos diferentes disponíveis. Aparentemente o melhor é o **modnet_photographic**, cujo já está selecionado por padrão. 

#@markdown Caso queira testar outro modelo, o **u2net** também é bom

#@markdown *Sim, nem sempre corta certinho, mas é só usar a imagem original com fundo para ajustar no Photoshop*

model = 'modnet_photographic' #@param ['u2net', 'modnet_photographic', 'modnet_webcam', 'u2net_portrait', 'u2net + u2net_portrait']
sketch_color = 'Gray'
one_frame_delay = True
one_frame_delay_threshold = 0.1
mirror_foreground = False
mirror_background = False
gray_background = False
bokeh_background = False
bokeh_prcnt = 5
bokeh_gamma = 5
keep_aspect_background = False
loop_reverse_background = False
copy_audio = True
bg_mode_max_w = 1920
chroma_thresholds = [0.5,0.25]
sketch_color = sketch_color.lower()

%cd /content
fg_dir = '/content/U-2-Net/test_data/test_images'
mask_dir = '/content/U-2-Net/test_data/u2net_results'
bg_dir = '/content/bg_frames'
result_dir = '/content/out_frames'
portrait_in_dir = '/content/U-2-Net/test_data/test_portrait_images/portrait_im'
portrait_out_dir = '/content/U-2-Net/test_data/test_portrait_images/portrait_results'
!rm -rf $fg_dir
!mkdir -p $fg_dir
!rm -rf $mask_dir
!mkdir -p $mask_dir
!rm -rf $bg_dir
!mkdir -p $bg_dir
!rm -rf $result_dir
!mkdir -p $result_dir
!rm -rf $portrait_in_dir
!mkdir -p $portrait_in_dir
!rm -rf $portrait_out_dir
!mkdir -p $portrait_out_dir

import imageio
imageio.plugins.freeimage.download()
import cv2
import numpy as np
import os
from time import time
import io
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from MODNet.src.models.modnet import MODNet
import warnings
warnings.filterwarnings("ignore")

torch_transforms = transforms.Compose(
  [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
  ]
)

def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]

def crop_resize(im, size, crop=False):
  if im.shape[:2] == size:
    return im
  if size[0]<im.shape[0] or size[1]<im.shape[1]:
    interp = cv2.INTER_AREA
  else:
    interp = cv2.INTER_CUBIC
  if not crop:
    return np.clip(cv2.resize(im, size[::-1], interpolation=interp),0,1)
  ratio = max(size[0]/im.shape[0], size[1]/im.shape[1])
  im = np.clip(cv2.resize(im, (int(np.ceil(im.shape[1]*ratio)), int(np.ceil(im.shape[0]*ratio))), interpolation=interp),0,1)
  return im[(im.shape[0]-size[0])//2:(im.shape[0]-size[0])//2+size[0], (im.shape[1]-size[1])//2:(im.shape[1]-size[1])//2+size[1]]

ref_size = 512
def modnet_matting(modnet, im):
  im_h, im_w = im.shape[:2]
  im_tensor = torch_transforms(im).float()
  im_tensor = im_tensor[None, :, :, :].cuda()
  
  if max(im_h, im_w) < ref_size or min(im_h, im_w) > ref_size:
    if im_w >= im_h:
      im_rh = ref_size
      im_rw = int(im_w / im_h * ref_size)
    elif im_w < im_h:
      im_rw = ref_size
      im_rh = int(im_h / im_w * ref_size)
  else:
    im_rh = im_h
    im_rw = im_w
        
  im_rw = im_rw - im_rw % 32
  im_rh = im_rh - im_rh % 32
  
  if im_h!=im_rh or im_w!=im_rw:
    im_tensor = F.interpolate(im_tensor, size=(im_rh, im_rw), mode='area')
  
  with torch.no_grad():
    _, _, matte_tensor = modnet(im_tensor, True)
  matte_tensor = F.interpolate(matte_tensor, size=(im_h, im_w), mode='area')
  matte_tensor = matte_tensor.repeat(1, 3, 1, 1)
  return matte_tensor[0].data.cpu().numpy().transpose(1, 2, 0)

grand_start = time()
start = time()
try:
    fg_now = imageio.imread('/content/foreground')
    fg_now = fix_dims(fg_now)
    imageio.imwrite(fg_dir+'/frame_%05d.png'%1, fg_now, format='PNG-FI')
    fg_now = fg_now/255
except Exception:
    !ffmpeg $fg_time_params -i /content/foreground $fg_dir/frame_%05d.png
    fg_now = imageio.imread(fg_dir+'/frame_%05d.png'%1, format='PNG-FI')
fg_files = [x for x in sorted(os.listdir(fg_dir)) if x.endswith('.png')]
prepare_time = time()-start

start = time()
have_u2_mask = False
if model.startswith('u2net') and (model!='u2net_portrait' or background_url is None or '://' in background_url or background_url=='foreground'):
  %cd /content/U-2-Net
  !python /content/U-2-Net/u2net_test.py
  have_u2_mask = True
elif model.startswith('modnet'):
  %cd /content/MODNet
  modnet = MODNet(backbone_pretrained=False)
  modnet = nn.DataParallel(modnet).cuda()
  modnet.load_state_dict(torch.load('/content/MODNet/pretrained/'+model+'_portrait_matting.ckpt'))
  modnet.eval()
mask_time = time()-start

blend_time = 0
is_fg = True
bg_files = []
iter_files = fg_files
rounds = 0

def preproc_bg(bg, fg_now):
  bg = crop_resize(bg, fg_now.shape[:2], crop=keep_aspect_background)
  if bokeh_background:
    if bokeh_gamma>1:
      bg = bg**bokeh_gamma
    radius = int(bokeh_prcnt/100*np.sqrt(bg.shape[0]*bg.shape[1])//2*2+1)
    if radius>1:
      bg = cv2.GaussianBlur(bg,(radius,radius),0)
    if bokeh_gamma>1:
      bg **= 1/bokeh_gamma
  if gray_background:
    bg = fix_dims(np.dot(bg, [0.2989, 0.5870, 0.1140]))
  if mirror_background:
    bg = np.fliplr(bg)
  return bg

def mat(in_dir, out_dir, bg_mode, orig_dir=None, fg_mirror=mirror_foreground):
  global prepare_time, blend_time, fg_now, is_fg, bg_files, iter_files, rounds
  rounds += 1
  start = time()
  if bg_mode == 'white':
    bg = np.full_like(fg_now, 1)
  elif bg_mode == 'black':
    bg = np.full_like(fg_now, 0)
  elif bg_mode == 'chroma green':
    bg = np.full_like(fg_now, [0,177/255,64/255])
  elif bg_mode == 'chroma blue':
    bg = np.full_like(fg_now, [0,71/255,187/255])
  elif background_url is None or '://' in background_url:
    try:
      bg = imageio.imread('/content/background')/255
      bg = fix_dims(bg)
      bg = preproc_bg(bg, fg_now)
    except Exception:
      print('!!!!2')
      !ffmpeg $bg_time_params -i /content/background $bg_dir/frame_%05d.png
  elif bg_mode not in ['transparent','foreground']:
    bg = np.full_like(fg_now, [int(background_url.lstrip('#')[i:i+2], 16)/255 for i in [0, 2, 4]])
  bg_files = [x for x in sorted(os.listdir(bg_dir)) if x.endswith('.png')]
  prepare_time += time()-start

  start = time()
  fg_plus = None
  mask_plus = None
  orig = None
  is_fg = len(fg_files)>1 or not bg_files
  if is_fg:
    iter_files = fg_files
  else:
    iter_files = bg_files
    fg_now = imageio.imread(in_dir+'/frame_%05d.png'%1, format='PNG-FI')/255
    if have_u2_mask:
      mask = imageio.imread(mask_dir+'/frame_%05d.png'%1, format='PNG-FI')/255
    elif model.startswith('modnet'):
      mask = modnet_matting(modnet, fg_now)
    else:
      mask = np.ones_like(fg_now)
    if orig_dir is not None:
      orig = imageio.imread(orig_dir+'/frame_%05d.png'%1, format='PNG-FI')/255
    if mirror_foreground:
      fg_now = np.fliplr(fg_now)
      mask = np.fliplr(mask)
      if orig is not None:
        orig = np.fliplr(orig)

  j = -1
  j_direction = 1
  for i,file in enumerate(iter_files):
      if is_fg:
        if one_frame_delay and i>0 and i<len(fg_files)-1:
          fg_now = fg_plus
          mask_minus = mask_now
          mask_now = mask_plus
          if fg_now is None:
            fg_now = imageio.imread(in_dir+'/'+file,format='PNG-FI')/255
          if have_u2_mask:
            if mask_now is None:
              mask_now = imageio.imread(mask_dir+'/'+file,format='PNG-FI')/255
            mask_plus = imageio.imread(mask_dir+'/'+fg_files[i+1],format='PNG-FI')/255
          elif model.startswith('modnet'):
            if mask_now is None:
              mask_now = modnet_matting(modnet, fg_now)
            fg_plus = imageio.imread(in_dir+'/'+fg_files[i+1],format='PNG-FI')/255
            mask_plus = modnet_matting(modnet, fg_plus)
          else:
            if mask_now is None:
              mask_now = np.ones_like(fg_now)
            mask_plus = np.ones_like(fg_now)          
          cond = (np.abs(mask_plus-mask_minus)<=one_frame_delay_threshold) & (np.abs(mask_now-mask_minus)>one_frame_delay_threshold) & (np.abs(mask_now-mask_plus)>one_frame_delay_threshold)
          mask = mask_now*(1-cond) + (mask_minus+mask_plus)/2*cond
        else:
          fg_now = imageio.imread(in_dir+'/'+file,format='PNG-FI')/255
          if have_u2_mask:
            mask_now = imageio.imread(mask_dir+'/'+file,format='PNG-FI')/255
          elif model.startswith('modnet'):
            mask_now = modnet_matting(modnet, fg_now)
          else:
            mask_now = np.ones_like(fg_now)
          mask = mask_now

        if orig_dir is not None:  
          orig = imageio.imread(orig_dir+'/'+file,format='PNG-FI')/255
        if fg_mirror:
            fg_now = np.fliplr(fg_now)
            mask = np.fliplr(mask)
            if orig is not None:
              orig = np.fliplr(orig)
        if bg_mode=='foreground':
          if orig is not None:
            bg = orig
          else:
            bg = fg_now
        elif bg_files:
          if loop_reverse_background: 
            j += j_direction
            if j>=len(bg_files):
              j = 2*len(bg_files)-j-1
              j_direction = -1
            elif j<0:
              j = 0
              j_direction = 1
          else:
            j = i%len(bg_files)
          bg = imageio.imread(bg_dir+'/'+bg_files[j],format='PNG-FI')/255
      
      else:
        bg = imageio.imread(bg_dir+'/'+file,format='PNG-FI')/255
      if bg_files:
        bg = preproc_bg(bg, fg_now)
      if bg_mode == 'transparent':
        im = np.dstack([fg_now,mask[:,:,0]])
      else:
        fg = fg_now
        if orig is not None:
          if sketch_color=='foreground':
            fg = 1-(1-fg)*(1-orig)
          elif 'tint' in sketch_color:
            non_black_mask = np.any(mask != [0, 0, 0], axis=-1)
            colors = (orig*mask)[non_black_mask]
            chroma = np.max(colors, axis=-1)-np.min(colors, axis=-1)
            chroma_thresholds.sort(reverse=True)
            if sketch_color=='tint fill':
              chroma_thresholds.append(0)
            else:
              color = np.array([0,0,0])
            for threshold in chroma_thresholds:
              cond = chroma>=threshold
              if np.any(cond):
                unique, counts = np.unique(colors[cond], axis=0, return_counts=True)
                color = unique[np.argmax(counts)]
                break
            if sketch_color=='tint outline':
              fg = 1-(1-fg)*(1-color)
            elif sketch_color=='tint fill':
              fg = fg*color
        im = bg*(1-mask)+fg*mask
      if not is_fg and bg_mode_max_w and fg_now.shape[1]>bg_mode_max_w:
        h = int(np.round(fg_now.shape[1]/fg_now.shape[0]*w))
        im = crop_resize(im, (bg_mode_max_w,h), crop=keep_aspect_background)
      imageio.imwrite(out_dir+'/'+file, np.uint8(im*255), compression=1 if len(fg_files)>1 or bg_files else 9, format='PNG-FI')
      print('%d/%d (%d)'%(i+1,len(iter_files),rounds))
  blend_time += time()-start

out_dir = result_dir
bg_mode = background_url
if 'u2net_portrait' in model:
  out_dir = portrait_in_dir
  bg_mode = 'white' 
if model!='u2net_portrait':
  mat(fg_dir, out_dir, bg_mode, fg_mirror=False if '+' in model else mirror_foreground)
elif 'u2net_portrait' in model:
  if background_url=='foreground' or sketch_color!='gray':
    !cp $fg_dir/* $portrait_in_dir
  else:
    !mv $fg_dir/* $portrait_in_dir
if 'u2net_portrait' in model:
  %cd /content/U-2-Net
  start = time()
  !python /content/U-2-Net/u2net_portrait_test.py
  blend_time += time()-start
  if background_url is None or '://' in background_url or background_url=='foreground' or sketch_color!='gray':
    mat(portrait_out_dir, result_dir, background_url, orig_dir=fg_dir if background_url=='foreground' or sketch_color!='gray' else None)
  else:
    !mv $portrait_out_dir/* $result_dir

start = time()
from IPython.display import HTML, clear_output, Image
from base64 import b64encode
import shutil
!rm -f /content/final.mp4
!rm -f /content/final.png
if len(fg_files)>1 or len(bg_files)>1:
  if is_fg:
    with imageio.get_reader('/content/foreground', format='mp4') as reader:
      fps = reader.get_meta_data()['fps']
    if copy_audio:
      !ffmpeg -framerate $fps -i $result_dir/frame_%05d.png $fg_time_params -i /content/foreground -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
    else:
      !ffmpeg -framerate $fps -i $result_dir/frame_%05d.png -c:v libx264 -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
  else:
    with imageio.get_reader('/content/background', format='mp4') as reader:
      fps = reader.get_meta_data()['fps']
    if copy_audio:
      !ffmpeg -framerate $fps -i $result_dir/frame_%05d.png $bg_time_params -i /content/background -c:v libx264 -c:a aac -map 0:v -map 1:a? -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
    else:
      !ffmpeg -framerate $fps -i $result_dir/frame_%05d.png -c:v libx264 -c:a aac -vf "crop=trunc(iw/2)*2:trunc(ih/2)*2" -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
  #video can be downloaded from /content/final.mp4
  save_time = time()-start
  total_time = time()-grand_start  
  clear_output()
  with open('/content/final.mp4', 'rb') as f:
    data_url = "data:video/mp4;base64," + b64encode(f.read()).decode()
  display(HTML("""
  <video width=600 controls autoplay loop>
        <source src="%s" type="video/mp4">
  </video>""" % data_url))
else:
  shutil.move(out_dir+'/frame_%05d.png'%1, '/content/final.png')
  #image can be downloaded from /content/final.png
  save_time = time()-start
  total_time = time()-grand_start
  clear_output()
  display(Image('/content/final.png', width=600))
if model.startswith('u2net'):
    print('frames=%i prepare=%i mask=%i blend=%i save=%i total=%i'%(len(iter_files), prepare_time, mask_time, blend_time, save_time, total_time))
else:
    print('frames=%i prepare=%i mask+blend=%i save=%i total=%i'%(len(iter_files), prepare_time, mask_time+blend_time, save_time, total_time))


In [ ]:
#@title Baixe a imagem

print() #see https://github.com/googlecolab/colabtools/issues/468
from google.colab import files
if os.path.exists('/content/final.mp4'):
  files.download('/content/final.mp4') #fails for Firefox private window
else:
  files.download('/content/final.png')